In [ ]:
!pip install plotly

In [ ]:
pip install geopandas shapely

In [ ]:
import pandas as pd
import plotly.express as px
import geopandas as gpd
from shapely.geometry import Point

## Create an interactive visualization that shows all crime in nyc

In [ ]:
url = "https://data.cityofnewyork.us/api/views/8h9b-rp9u/rows.csv?accessType=DOWNLOAD"
NYPD_arrests_df = pd.read_csv(url) # was too large to host directly in github
NYPD_arrests_df

#NYPD_arrests_df = pd.read_csv('NYPD_Arrests_Data__Historic.csv')

In [ ]:
NYPD_arrests_df

In [ ]:
# shorten the df to increase processing speed during testing
# remove when not testing
#NYPD_arrests_df = NYPD_arrests_df[0:1500]

In [ ]:
print(NYPD_arrests_df.columns)

In [ ]:
# Convert ARREST_DATE to datetime in order to select a year
NYPD_arrests_df["ARREST_DATE"] = pd.to_datetime(NYPD_arrests_df["ARREST_DATE"], errors="coerce")
year = 2023
NYPD_arrests_df = NYPD_arrests_df[NYPD_arrests_df["ARREST_DATE"].dt.year == year]

In [ ]:
# Unabbreviate Offense level
NYPD_arrests_df["LAW_CAT_CD"] = NYPD_arrests_df["LAW_CAT_CD"].replace({
    "F": "Felony",
    "M": "Misdemeanor",
    "V": "Violation",
    "9": "Other",
    "I": "Other"
}).fillna("Other")

In [ ]:
# Create a scatter map using Plotly
fig = px.scatter_mapbox(
    NYPD_arrests_df,
    lat="Latitude",
    lon="Longitude",
    color="LAW_CAT_CD",  # Colors dots by level of offense
    hover_data=["OFNS_DESC", "ARREST_DATE", "ARREST_BORO"],
    zoom=12,
    height=2000,
    title="Sample of NYC Arrests in {year} by Offense Level"
)

# openstreetmap
fig.update_layout(mapbox_style="open-street-map")

fig.show()

In [ ]:
# number of arressts in selected year
len(NYPD_arrests_df)

## Creating an interactive visualization that highlights prevelance of public facilities
Highlights crime outside of a specific distance from public facilities

In [ ]:
facilities_df = pd.read_csv("./Data/raw/Facilities_Database.csv")
facilities_df

In [ ]:
public_facilities_df = facilities_df[facilities_df["optype"] == "Public"]
public_facilities_df

In [ ]:
# Set up the data to create a visualization that determines if an arrest was near public facilities

arrest_df = NYPD_arrests_df#[:1000] #selecting a specific bunch can be used to test and shorten load time
facilities_df = facilities_df#[:1000]

# Convert to geodataframes
arrests_gdf = gpd.GeoDataFrame(
    arrest_df,
    geometry=gpd.points_from_xy(arrest_df["Longitude"], arrest_df["Latitude"]),
    crs="EPSG:4326"  # standard lat long system
).to_crs(epsg=2263)  # NYC coordinate system in feet

facilities_gdf = gpd.GeoDataFrame(
    public_facilities_df,
    geometry=gpd.points_from_xy(public_facilities_df["longitude"], public_facilities_df["latitude"]),
    crs="EPSG:4326"
).to_crs(epsg=2263)

# buffer zone around a facility
buffer_dist_ft = 1000  # distance in ft
fac_buffer = facilities_gdf.buffer(buffer_dist_ft)

# merge the buffer zones into grouped circles (the circles each maintain their independent shape)
combined_buffer = fac_buffer.unary_union

# keep only arrests outside of the buffer area
arrests_outside_buffer = arrests_gdf[~arrests_gdf.geometry.within(combined_buffer)]

# Convert back to WGS84 (standard lat long system) for plotting
arrests_outside_buffer = arrests_outside_buffer.to_crs(epsg=4326)

In [ ]:
#create a visualization that determines if an arrest was near public facilities

# Plot arrests outside buffer
fig = px.scatter_mapbox(
    arrests_outside_buffer,
    lat=arrests_outside_buffer.geometry.y,
    lon=arrests_outside_buffer.geometry.x,
    color="LAW_CAT_CD",
    hover_data=["OFNS_DESC", "ARREST_DATE"],
    zoom=10,
    height=700,
    title=f"{year} Arrests NOT Within {buffer_dist_ft}ft of a Public Facility"
)
fig.update_layout(mapbox_style="open-street-map")
fig.show()

In [ ]:
# save arrests outside buffer df as a csv
#arrests_outside_buffer.to_csv("arrests_outside_buffer.csv", index=False)

## Using buffer area to calculate public facility per capita effect on the rate of crime by neighborhood

In [ ]:
# Load NTA (neighborhood tabulation area) boundaries and project to feet
# Neighborhood tabulation area is neighborhood boundaries
nta = gpd.read_file("./Data/raw/2020_neighborhood_tabulation_areas.geojson").to_crs(epsg=2263)

# Create buffers around public facilities
facilities_gdf = gpd.GeoDataFrame(
    public_facilities_df,
    geometry=gpd.points_from_xy(public_facilities_df["longitude"], public_facilities_df["latitude"]),
    crs="EPSG:4326"
).to_crs(epsg=2263)
facility_buffer = facilities_gdf.buffer(buffer_dist_ft).unary_union  # buffer
 
# Get part of each NTA that overlaps with the buffer
nta_clip = gpd.overlay(nta[["nta2020", "geometry"]], gpd.GeoDataFrame(geometry=[facility_buffer], crs=nta.crs), how="intersection")
nta_clip["buffer_area"] = nta_clip.geometry.area

# Total area of each NTA
nta["total_area"] = nta.geometry.area

# Sum bufered area per NTA and merge with total
buffered_area = nta_clip.groupby("nta2020")["buffer_area"].sum().reset_index()
nta = nta.merge(buffered_area, on="nta2020", how="left")
nta["buffer_area"] = nta["buffer_area"].fillna(0)
nta["non_buffer_area"] = nta["total_area"] - nta["buffer_area"]

# Set up arrests GeoDataFrame
arrests = gpd.GeoDataFrame(
    NYPD_arrests_df.dropna(subset=["Longitude", "Latitude"]),
    geometry=gpd.points_from_xy(NYPD_arrests_df["Longitude"], NYPD_arrests_df["Latitude"]),
    crs="EPSG:4326"
).to_crs(epsg=2263)

# Attach each arrest to a neighborhood
arrests = gpd.sjoin(arrests, nta[["nta2020", "geometry"]], how="inner", predicate="within")

# Notate arrests inside the buffer zone
arrests["in_buffer"] = arrests.geometry.within(facility_buffer)

# Count arrests inside and outside the buffer for each NTA
counts = arrests.groupby(["nta2020", "in_buffer"]).size().unstack(fill_value=0).reset_index()
counts.columns = ["nta2020", "outside_arrests", "inside_arrests"]

# Merge counts with area data
nta = nta.merge(counts, on="nta2020", how="left")
nta["inside_arrests"] = nta["inside_arrests"].fillna(0)
nta["outside_arrests"] = nta["outside_arrests"].fillna(0)

# Arrest density per square foot
nta["inside_density"] = nta["inside_arrests"] / nta["buffer_area"]
nta["outside_density"] = nta["outside_arrests"] / nta["non_buffer_area"]

# Final view
nta[["nta2020", "inside_arrests", "outside_arrests","buffer_area", "non_buffer_area","inside_density", "outside_density"]]

In [ ]:
nta

In [ ]:
nta.columns

In [ ]:
nta["density_diff"] = nta["inside_density"] - nta["outside_density"]
nta[['ntaname','density_diff']]

In [ ]:
borough_avg_diff = nta.groupby("boroname")["density_diff"].mean().reset_index()
borough_avg_diff.columns = ["borough", "avg_density_difference"]
borough_avg_diff

In [ ]:
#save nta as a geojson
#nta.to_file("nta.geojson", driver="GeoJSON")

# Can call it using the following: nta = gpd.read_file("nta.geojson")

## Evaluation of public facility effect on crime

Per square foot there is the above increase in crime inside the buffer (avg_density_difference). This increase is not extraordinarily high. But the consistency across boroughs is hard to ignore, and is a bit concerning. Many public facilities are built to combat issues so they will generally be built nearby the problems they intend to alleviate. Also the public facilities were not filtered. A closer examination of facilities by type could be useful. For instance, prisons count as a public facility here. Filtering by public facility could be helpful, but we were a bit concerned about p-hacking. So we decided to be broad. Using buffer with a radius of 1000 feet may also not be perfect as it was slightly arbitrarily decided on based on what seemed to visually make sense for the 2nd visualizationa above. This allowed for some crime outside of buffer zones to show while creating noticeable buffer zones. Also, crime being the only barometer of success is also an imperfect measure for the effectiveness of all public facilities.